In [1]:
from __future__ import print_function

#Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('C:/Users/StrikeWade/Desktop/DLML/Tensorflow Examples Models/mnist', one_hot=True)

import tensorflow as tf
from keras import backend as K

#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting C:/Users/StrikeWade/Desktop/DLML/Tensorflow Examples Models/mnist\train-images-idx3-ubyte.gz
Extracting C:/Users/StrikeWade/Desktop/DLML/Tensorflow Examples Models/mnist\train-labels-idx1-ubyte.gz
Extracting C:/Users/StrikeWade/Desktop/DLML/Tensorflow Examples Models/mnist\t10k-images-idx3-ubyte.gz
Extracting C:/Users/StrikeWade/Desktop/DLML/Tensorflow Examples Models/mnist\t10k-labels-idx1-ubyte.gz


Using TensorFlow backend.


In [2]:
#Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

#Network Parameters
n_hidden_1 = 256 #1st layer number of neurons
n_hidden_2 = 256 #2nd layer number of neurons
num_input = 784 #Mnist data input has 784 features (img_shape=28*28)
num_classes = 10 #Mnist total classes (0-9 digits)

#tf Graph input
x = tf.placeholder(dtype=tf.float32, shape=[None, num_input])
y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])

In [3]:
#Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [4]:
#Create model
def simple_neural_net(x):
    #Hidden layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    #Hidden layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    #Output fully connected layer with a neuron for each class
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return out_layer

In [5]:
#Construct model
logits = simple_neural_net(x)

#Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#Evaluate model(with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

#Init global variable
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [7]:
#Start trainning
with tf.Session() as sess:
    
    #Run init
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
        
        #Run optimization op (backprop)
        sess.run(train_op, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            #Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x, y: batch_y})
            print('Step: ', str(step), '\n    Minibatch Loss= {:.4f}'.format(loss), '\n    Training Accuracy= {:.3f}'.format(acc))
            
    print('-->Training Completed!!!')
    
    #Calculate accuracy for MNIST test images
    print('-->Testing Accuracy: {:.4f}'.format(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})))

Step:  100 
    Minibatch Loss= 307.6748 
    Training Accuracy= 0.883
Step:  200 
    Minibatch Loss= 155.1941 
    Training Accuracy= 0.906
Step:  300 
    Minibatch Loss= 80.9356 
    Training Accuracy= 0.875
Step:  400 
    Minibatch Loss= 69.4892 
    Training Accuracy= 0.867
Step:  500 
    Minibatch Loss= 29.8902 
    Training Accuracy= 0.898
-->Training Completed!!!
-->Testing Accuracy: 0.8435
